---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
# Load the data
# Reference: https://necromuralist.github.io/data_science/posts/extracting-dates-from-medical-data/
import pandas

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

data = pandas.Series(doc)
data.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
data.describe()

count                                                   500
unique                                                  500
top       1979 Family Psych History: Family History of S...
freq                                                      1
dtype: object

In [3]:
# 4 The Grammar
# 4.1 Cardinality
ZERO_OR_MORE = '*'
ONE_OR_MORE = "+"
ZERO_OR_ONE = '?'
EXACTLY_TWO = "{2}"
ONE_OR_TWO = "{1,2}"
EXACTLY_ONE = '{1}'

In [4]:
# 4.2 Groups and Classes
GROUP = r"({})"
NAMED = r"(?P<{}>{})"
CLASS = "[{}]"
NEGATIVE_LOOKAHEAD = "(?!{})"
NEGATIVE_LOOKBEHIND = "(?<!{})"
POSITIVE_LOOKAHEAD = "(?={})"
POSITIVE_LOOKBEHIND = "(?<={})"
ESCAPE = "\{}"

In [5]:
# 4.3 Numbers
DIGIT = r"\d"
ONE_DIGIT  = DIGIT + EXACTLY_ONE
ONE_OR_TWO_DIGITS = DIGIT + ONE_OR_TWO
NON_DIGIT = NEGATIVE_LOOKAHEAD.format(DIGIT)
TWO_DIGITS = DIGIT + EXACTLY_TWO
THREE_DIGITS = DIGIT + "{3}"
EXACTLY_TWO_DIGITS = DIGIT + EXACTLY_TWO + NON_DIGIT
FOUR_DIGITS = DIGIT + r"{4}" + NON_DIGIT

In [6]:
# 4.4 String Literals
SLASH = r"/"
OR = r'|'
LOWER_CASE = "a-z"
SPACE = "\s"
DOT = "."
DASH = "-"
COMMA = ","
PUNCTUATION = CLASS.format(DOT + COMMA + DASH)
EMPTY_STRING = ""

In [7]:
# 4.5 Dates
# These are parts to build up the date-expressions.
MONTH_SUFFIX = (CLASS.format(LOWER_CASE) + ZERO_OR_MORE
                + CLASS.format(SPACE + DOT + COMMA + DASH) + ONE_OR_TWO)
MONTH_PREFIXES = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split()
MONTHS = [month + MONTH_SUFFIX for month in MONTH_PREFIXES]
MONTHS = GROUP.format(OR.join(MONTHS))
DAY_SUFFIX = CLASS.format(DASH + COMMA + SPACE) + ONE_OR_TWO
DAYS = ONE_OR_TWO_DIGITS + DAY_SUFFIX
YEAR = FOUR_DIGITS

In [8]:
# This is for dates like Mar 21st, 2009, those with suffixes on the days.
CONTRACTED = (ONE_OR_TWO_DIGITS
              + LOWER_CASE
              + EXACTLY_TWO
              )
CONTRACTION = NAMED.format("contraction",
                           MONTHS
                           + CONTRACTED
                           + DAY_SUFFIX
                           + YEAR)

In [9]:
# This is for dates that have no days in them, like May 2009.
NO_DAY_BEHIND = NEGATIVE_LOOKBEHIND.format(DIGIT + SPACE)
NO_DAY = NAMED.format("no_day", NO_DAY_BEHIND + MONTHS + YEAR)

In [10]:
# This is for the most common form (that I use) - May 21, 2017.
WORDS = NAMED.format("words", MONTHS + DAYS + YEAR)

In [11]:
BACKWARDS = NAMED.format("backwards", ONE_OR_TWO_DIGITS + SPACE + MONTHS + YEAR)

In [12]:
slashed = SLASH.join([ONE_OR_TWO_DIGITS,
                      ONE_OR_TWO_DIGITS,
                      EXACTLY_TWO_DIGITS])
dashed = DASH.join([ONE_OR_TWO_DIGITS,
                    ONE_OR_TWO_DIGITS,
                    EXACTLY_TWO_DIGITS])
TWENTIETH_CENTURY = NAMED.format("twentieth",
                                 OR.join([slashed, dashed]))

In [13]:
NUMERIC = NAMED.format("numeric",
                       SLASH.join([ONE_OR_TWO_DIGITS,
                                   ONE_OR_TWO_DIGITS,
                                   FOUR_DIGITS]))

In [14]:
NO_PRECEDING_SLASH = NEGATIVE_LOOKBEHIND.format(SLASH)
NO_PRECEDING_SLASH_DIGIT = NEGATIVE_LOOKBEHIND.format(CLASS.format(SLASH + DIGIT))
NO_ONE_DAY = (NO_PRECEDING_SLASH_DIGIT
              + ONE_DIGIT
              + SLASH
              + FOUR_DIGITS)
NO_TWO_DAYS = (NO_PRECEDING_SLASH
               + TWO_DIGITS
               + SLASH
               + FOUR_DIGITS)
NO_DAY_NUMERIC = NAMED.format("no_day_numeric",
                              NO_ONE_DAY
                              + OR
                              + NO_TWO_DAYS
                              )

In [15]:
CENTURY = GROUP.format('19' + OR + "20") + TWO_DIGITS
DIGIT_SLASH = DIGIT + SLASH
DIGIT_DASH = DIGIT + DASH
DIGIT_SPACE = DIGIT + SPACE
LETTER_SPACE = CLASS.format(LOWER_CASE) + SPACE
COMMA_SPACE = COMMA + SPACE
YEAR_PREFIX = NEGATIVE_LOOKBEHIND.format(OR.join([
    DIGIT_SLASH,
    DIGIT_DASH,
    DIGIT_SPACE,
    LETTER_SPACE,
    COMMA_SPACE,
]))

YEAR_ONLY = NAMED.format("year_only",
                         YEAR_PREFIX + CENTURY
)

In [16]:
IN_PREFIX = POSITIVE_LOOKBEHIND.format(CLASS.format('iI') + 'n' + SPACE) + CENTURY
SINCE_PREFIX = POSITIVE_LOOKBEHIND.format(CLASS.format("Ss") + 'ince' + SPACE) + CENTURY
AGE = POSITIVE_LOOKBEHIND.format("Age" + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
AGE_COMMA = POSITIVE_LOOKBEHIND.format("Age" + COMMA + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
OTHERS = ['delivery', "quit", "attempt", "nephrectomy", THREE_DIGITS]
OTHERS = [POSITIVE_LOOKBEHIND.format(label + SPACE) + CENTURY for label in OTHERS]
OTHERS = OR.join(OTHERS)
LEFTOVERS_PREFIX = OR.join([IN_PREFIX, SINCE_PREFIX, AGE, AGE_COMMA]) + OR + OTHERS
LEFTOVERS = NAMED.format("leftovers", LEFTOVERS_PREFIX)

In [17]:
DATE = NAMED.format("date", OR.join([NUMERIC,
                                     TWENTIETH_CENTURY,
                                     WORDS,
                                     BACKWARDS,
                                     CONTRACTION,
                                     NO_DAY,
                                     NO_DAY_NUMERIC,
                                     YEAR_ONLY,
                                     LEFTOVERS]))

In [18]:
def twentieth_century(date):
    """adds a 19 to the year

    Args:
     date (re.Regex): Extracted date
    """
    month, day, year = date.group(1).split(SLASH)
    year = "19{}".format(year)
    return SLASH.join([month, day, year])

In [19]:
def take_two(line):
    match = re.search(TWENTIETH_CENTURY, line)
    if match:
        return twentieth_century(match)
    return line

In [20]:
def extract_and_count(expression, data, name):
    """extract all matches and report the count

    Args:
     expression (str): regular expression to match
     data (pandas.Series): data with dates to extratc
     name (str): name of the group for the expression

    Returns:
     tuple (pandas.Series, int): extracted dates, count
    """
    extracted = data.str.extractall(expression)[name]
    count = len(extracted)
    print("'{}' matched {} rows".format(name, count))
    return extracted, count

In [21]:
numeric, numeric_count = extract_and_count(NUMERIC, data, 'numeric')
# 'numeric' matched 25 rows
twentieth, twentieth_count = extract_and_count(TWENTIETH_CENTURY, data, 'twentieth')
# 'twentieth' matched 100 rows
words, words_count = extract_and_count(WORDS, data, 'words')
# 'words' matched 34 rows
backwards, backwards_count = extract_and_count(BACKWARDS, data, 'backwards')
# 'backwards' matched 69 rows
contraction_data, contraction = extract_and_count(CONTRACTION, data, 'contraction')
# 'contraction' matched 0 rows
no_day, no_day_count = extract_and_count(NO_DAY, data, 'no_day')
# 'no_day' matched 115 rows
no_day_numeric, no_day_numeric_count = extract_and_count(NO_DAY_NUMERIC, data,
                                                         "no_day_numeric")
# 'no_day_numeric' matched 112 rows
year_only, year_only_count = extract_and_count(YEAR_ONLY, data, "year_only")
# 'year_only' matched 15 rows
leftovers, leftovers_count = extract_and_count(LEFTOVERS, data, "leftovers")
# 'leftovers' matched 30 rows

'numeric' matched 25 rows
'twentieth' matched 100 rows
'words' matched 34 rows
'backwards' matched 69 rows
'contraction' matched 0 rows
'no_day' matched 115 rows
'no_day_numeric' matched 112 rows
'year_only' matched 15 rows
'leftovers' matched 30 rows


In [22]:
found = data.str.extractall(DATE)
total_found = len(found.date)

print("Total Found: {}".format(total_found))
print("Remaining: {}".format(len(data) - total_found))
print("Discrepancy: {}".format(total_found - (numeric_count
                                              + twentieth_count
                                              + words_count
                                              + backwards_count
                                              + contraction
                                              + no_day_count
                                              + no_day_numeric_count
                                              + year_only_count
                                              + leftovers_count)))

Total Found: 500
Remaining: 0
Discrepancy: 0


In [23]:
# Total Found: 500
# Remaining: 0
# Discrepancy: 0

In [24]:
missing = [label for label in data.index if label not in found.index.levels[0]]
try:
    print(missing[0], data.loc[missing[0]])
except IndexError:
    print("all rows matched")

all rows matched


In [25]:
# all rows matched

In [26]:
def clean(source, expression, replacement, sample=5):
    """applies the replacement to the source

    as a side-effect shows sample rows before and after

    Args:
     source (pandas.Series): source of the strings
     expression (str): regular expression to match what to replace
     replacement: function or expression to replace the matching expression
     sample (int): number of randomly chosen examples to show

    Returns:
     pandas.Series: the source with the replacement applied to it
    """
    print("Random Sample Before:")
    print(source.sample(sample))
    cleaned = source.str.replace(expression, replacement)
    print("\nRandom Sample After:")
    print(cleaned.sample(sample))
    print("\nCount of cleaned: {}".format(len(cleaned)))
    assert len(source) == len(cleaned)
    return cleaned

In [27]:
def clean_punctuation(source, sample=5):
    """removes punctuation

    Args:
     source (pandas.Series): data to clean
     sample (int): size of sample to show

    Returns:
     pandas.Series: source with punctuation removed
    """
    print("Cleaning Punctuation")
    if any(source.str.contains(PUNCTUATION)):
        source = clean(source, PUNCTUATION, EMPTY_STRING)
    return source

In [28]:
LONG_TO_SHORT = dict(January="Jan",
                     February="Feb",
                     March="Mar",
                     April="Apr",
                     May="May",
                     June="Jun",
                     July="Jul",
                     August="Aug",
                     September="Sep",
                     October="Oct",
                     November="Nov",
                     December="Dec")

# it turns out there are spelling errors in the data so this has to be fuzzy
LONG_TO_SHORT_EXPRESSION = OR.join([GROUP.format(month)
                                    + CLASS.format(LOWER_CASE)
                                    + ZERO_OR_MORE
                                    for month in LONG_TO_SHORT.values()])

def long_month_to_short(match):
    """convert long month to short

    Args:
     match (re.Match): object matching a long month

    Returns:
     str: shortened version of the month
    """
    return match.group(match.lastindex)

In [29]:
def convert_long_months_to_short(source, sample=5):
    """convert long month names to short

    Args:
     source (pandas.Series): data with months
     sample (int): size of sample to show

    Returns:
     pandas.Series: data with short months
    """
    return clean(source,
                 LONG_TO_SHORT_EXPRESSION,
                 long_month_to_short)

In [30]:
def add_month_date(match):
    """adds 01/01 to years

    Args:
     match (re.Match): object that only matched a 4-digit year

    Returns:
     str: 01/01/YYYY
    """
    return "01/01/" + match.group()


In [31]:
def add_january_one(source):
    """adds /01/01/ to year-only dates

    Args:
     source (pandas.Series): data with the dates

    Returns:
     pandas.Series: years in source with /01/01/ added
    """
    return clean(source, YEAR_ONLY, add_month_date)

In [32]:
two_digit_expression = GROUP.format(ONE_OR_TWO_DIGITS) + POSITIVE_LOOKAHEAD.format(SLASH)

def two_digits(match):
    """add a leading zero if needed

    Args:
     match (re.Match): match with one or two digits

    Returns:
     str: the matched string with leading zero if needed
    """
    # for some reason the string-formatting raises an error if it's a string
    # so cast it to an int
    return "{:02}".format(int(match.group()))

In [33]:
def clean_two_digits(source, sample=5):
    """makes sure source has two-digits

    Args:
     source (pandas.Series): data with digit followed by slash
     sample (int): number of samples to show

    Returns:
     pandas.Series: source with digits coerced to two digits
    """
    return clean(source, two_digit_expression, two_digits, sample)

In [34]:
def clean_two_digits_isolated(source, sample=5):
    """cleans two digits that are standalone

    Args:
     source (pandas.Series): source of the data
     sample (int): number of samples to show

    Returns:
     pandas.Series: converted data
    """
    return clean(source, ONE_OR_TWO_DIGITS, two_digits, sample)

In [35]:
digits = ("{:02}".format(month) for month in range(1, 13))
MONTH_TO_DIGITS = dict(zip(MONTH_PREFIXES, digits))
SHORT_MONTHS_EXPRESSION = OR.join((GROUP.format(month) for month in MONTH_TO_DIGITS))
def month_to_digits(match):
    """converts short month to digits

    Args:
     match (re.Match): object with short-month

    Returns:
     str: month as two-digit number (e.g. Jan -> 01)
    """
    return MONTH_TO_DIGITS[match.group()]

In [36]:
def convert_short_month_to_digits(source, sample=5):
    """converts three-letter months to two-digits

    Args:
     source (pandas.Series): data with three-letter months
     sample (int): number of samples to show

    Returns:
     pandas.Series: source with short-months coverted to digits
    """
    return clean(source,
                 SHORT_MONTHS_EXPRESSION,
                 month_to_digits,
                 sample)

In [37]:
def clean_months(source, sample=5):
    """clean up months (which start as words)

    Args:
     source (pandas.Series): source of the months
     sample (int): number of random samples to show
    """
    cleaned = clean_punctuation(source)

    print("Converting long months to short")
    cleaned = clean(cleaned,
                    LONG_TO_SHORT_EXPRESSION,
                    long_month_to_short, sample)

    print("Converting short months to digits")
    cleaned = clean(cleaned,
                    SHORT_MONTHS_EXPRESSION,
                    month_to_digits, sample)
    return cleaned

In [38]:
def frame_to_series(frame, index_source, samples=5):
    """re-combines data-frame into a series

    Args:
     frame (pandas.DataFrame): frame with month, day, year columns
     index_source (pandas.series): source to copy index from
     samples (index): number of random entries to print when done

    Returns:
     pandas.Series: series with dates as month/day/year
    """
    combined = frame.month + SLASH + frame.day + SLASH + frame.year
    combined.index = index_source.index
    print(combined.sample(samples))
    return combined

In [39]:
year_only_cleaned = add_january_one(year_only)

Random Sample Before:
     match
496  0        2006
472  0        2010
486  0        1973
495  0        1979
497  0        2008
Name: year_only, dtype: object

Random Sample After:
     match
481  0        01/01/1974
486  0        01/01/1973
473  0        01/01/1975
466  0        01/01/1981
472  0        01/01/2010
Name: year_only, dtype: object

Count of cleaned: 15


In [40]:
# Random Sample Before:
#      match
# 472  0        2010
# 495  0        1979
# 497  0        2008
# 481  0        1974
# 486  0        1973
# Name: year_only, dtype: object

# Random Sample After:
#      match
# 495  0        01/01/1979
# 470  0        01/01/1983
# 462  0        01/01/1988
# 481  0        01/01/1974
# 480  0        01/01/2013
# Name: year_only, dtype: object

# Count of cleaned: 15

In [41]:
leftovers_cleaned = add_january_one(leftovers)

Random Sample Before:
     match
488  0        1977
463  0        2014
494  0        2002
477  0        1994
474  0        1972
Name: leftovers, dtype: object

Random Sample After:
     match
475  0        01/01/2015
477  0        01/01/1994
488  0        01/01/1977
456  0        01/01/2000
494  0        01/01/2002
Name: leftovers, dtype: object

Count of cleaned: 30


In [42]:
# Random Sample Before:
#      match
# 487  0        1992
# 477  0        1994
# 498  0        2005
# 488  0        1977
# 484  0        2004
# Name: leftovers, dtype: object

# Random Sample After:
#      match
# 464  0        01/01/2016
# 455  0        01/01/1984
# 465  0        01/01/1976
# 475  0        01/01/2015
# 498  0        01/01/2005
# Name: leftovers, dtype: object

# Count of cleaned: 30

In [43]:
cleaned = pandas.concat([year_only_cleaned, leftovers_cleaned])
print(len(cleaned))

45


In [44]:
no_day_numeric_cleaned = clean_two_digits(no_day_numeric)

Random Sample Before:
     match
426  0        11/1984
417  0         8/2000
407  0         8/1999
389  0         2/2009
354  0         3/1993
Name: no_day_numeric, dtype: object

Random Sample After:
     match
363  0        12/1975
447  0        07/1985
351  0        08/1974
414  0        04/2004
344  0        06/2005
Name: no_day_numeric, dtype: object

Count of cleaned: 112


In [45]:
no_day_numeric_cleaned = clean(no_day_numeric_cleaned,
                               SLASH,
                               lambda m: "/01/")

Random Sample Before:
     match
366  0        07/2014
401  0        12/2014
452  0        03/2003
346  0        09/2005
380  0        07/1973
Name: no_day_numeric, dtype: object

Random Sample After:
     match
404  0        10/01/1986
405  0        03/01/1973
346  0        09/01/2005
407  0        08/01/1999
353  0        10/01/1997
Name: no_day_numeric, dtype: object

Count of cleaned: 112


In [46]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, no_day_numeric_cleaned])
assert len(cleaned) == no_day_numeric_count + original

In [47]:
print(len(cleaned))

157


In [48]:
no_day_cleaned = clean_months(no_day)

Cleaning Punctuation
Random Sample Before:
     match
269  0         July 1992
284  0          Jan 1987
327  0        April 1999
283  0          Feb 1977
256  0          Aug 1988
Name: no_day, dtype: object

Random Sample After:
     match
257  0          Sep 2015
295  0        March 1983
321  0         June 1999
276  0        April 1986
231  0          May 2016
Name: no_day, dtype: object

Count of cleaned: 115
Converting long months to short
Random Sample Before:
     match
260  0        February 2000
314  0         January 2007
332  0            June 1974
231  0             May 2016
302  0        November 2004
Name: no_day, dtype: object

Random Sample After:
     match
294  0        Feb 1983
271  0        Aug 2008
248  0        Jul 1995
254  0        Aug 1979
327  0        Apr 1999
Name: no_day, dtype: object

Count of cleaned: 115
Converting short months to digits
Random Sample Before:
     match
253  0        Feb 2016
274  0        Apr 1985
317  0        Mar 1975
326  0        Oc

In [49]:
no_day_cleaned = clean(no_day_cleaned,
                       SPACE + ONE_OR_MORE,
                       lambda match: "/01/")

Random Sample Before:
     match
319  0        07 1975
329  0        03 2000
229  0        06 2011
312  0        02 1989
294  0        02 1983
Name: no_day, dtype: object

Random Sample After:
     match
294  0        02/01/1983
280  0        07/01/1985
304  0        03/01/2002
261  0        10/01/1986
266  0        09/01/1999
Name: no_day, dtype: object

Count of cleaned: 115


In [50]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, no_day_cleaned])
print(len(cleaned))

272


In [51]:
assert len(cleaned) == no_day_count + original

In [52]:
frame = pandas.DataFrame(backwards.str.split().tolist(),
                         columns="day month year".split())
frame.head()

,day,month,year
0,24,Jan,2001
1,10,Sep,2004
2,26,May,1982
3,28,June,2002
4,06,May,1972


In [53]:
frame.day = clean_two_digits(frame.day)

Random Sample Before:
27    28
24    09
49    04
54    10
14    21
Name: day, dtype: object

Random Sample After:
7     30
46    04
22    30
35    21
20    18
Name: day, dtype: object

Count of cleaned: 69


In [54]:
frame.month = clean_months(frame.month)

Cleaning Punctuation
Converting long months to short
Random Sample Before:
22    May
51    May
10    Oct
64    Oct
38    Jan
Name: month, dtype: object

Random Sample After:
16    May
33    Aug
5     Oct
0     Jan
35    Oct
Name: month, dtype: object

Count of cleaned: 69
Converting short months to digits
Random Sample Before:
51    May
26    Jun
33    Aug
20    Oct
42    Oct
Name: month, dtype: object

Random Sample After:
18    10
53    06
44    10
45    01
10    10
Name: month, dtype: object

Count of cleaned: 69


In [55]:
backwards_cleaned = frame_to_series(frame, backwards)

     match
188  0        03/12/2004
137  0        02/10/1983
191  0        11/30/1972
136  0        02/11/1985
125  0        01/24/2001
dtype: object


In [56]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, backwards_cleaned])
assert len(cleaned) == original + backwards_count

In [57]:
print(len(cleaned))

341


In [58]:
frame = pandas.DataFrame(words.str.split().tolist(), columns="month day year".split())
print(frame.head())

      month  day  year
0     April  11,  1990
1       May  30,  2001
2       Feb  18,  1994
3  February  18,  1981
4  October.  11,  2013


In [59]:
frame.month = clean_months(frame.month)

Cleaning Punctuation
Random Sample Before:
6         July
30        July
4     October.
22         Nov
15        July
Name: month, dtype: object

Random Sample After:
22         Nov
20         Sep
26        June
16      August
7     December
Name: month, dtype: object

Count of cleaned: 34
Converting long months to short
Random Sample Before:
28      May
22      Nov
17    April
23     June
26     June
Name: month, dtype: object

Random Sample After:
15    Jul
19    Jul
18    Jul
20    Sep
29    Oct
Name: month, dtype: object

Count of cleaned: 34
Converting short months to digits
Random Sample Before:
2     Feb
29    Oct
31    Jun
24    May
26    Jun
Name: month, dtype: object

Random Sample After:
21    08
11    01
25    12
17    04
20    09
Name: month, dtype: object

Count of cleaned: 34


In [60]:
frame.day = clean_punctuation(frame.day)

Cleaning Punctuation
Random Sample Before:
28    15,
18    24,
12     23
26    25,
19    11,
Name: day, dtype: object

Random Sample After:
18    24
14    01
10    10
3     18
2     18
Name: day, dtype: object

Count of cleaned: 34


In [61]:
frame.head()

,month,day,year
0,04,11,1990
1,05,30,2001
2,02,18,1994
3,02,18,1981
4,10,11,2013


In [62]:
words_cleaned = frame_to_series(frame, words)

     match
201  0        12/23/1999
215  0        08/14/1981
199  0        01/24/1986
211  0        04/17/1992
200  0        07/26/1978
dtype: object


In [63]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, words_cleaned])
assert len(cleaned) == original + words_count
print(len(cleaned))

375


In [64]:
print(twentieth.iloc[21])
twentieth_cleaned = twentieth.str.replace(DASH, SLASH)
print(cleaned.iloc[21])

4-13-82
01/01/1991


In [65]:
frame = pandas.DataFrame(twentieth_cleaned.str.split(SLASH).tolist(),
                         columns=["month", "day", "year"])
print(frame.head())

  month day year
0    03  25   93
1     6  18   85
2     7   8   71
3     9  27   75
4     2   6   96


In [66]:
frame.month = clean_two_digits_isolated(frame.month)

Random Sample Before:
83     2
75     7
62    12
34     6
77     5
Name: month, dtype: object

Random Sample After:
5     07
20    02
63    04
56    07
2     07
Name: month, dtype: object

Count of cleaned: 100


In [67]:
frame.day = clean_two_digits_isolated(frame.day)

Random Sample Before:
31    24
80    05
8      7
49    06
15    24
Name: day, dtype: object

Random Sample After:
93    20
82    24
31    24
2     08
4     06
Name: day, dtype: object

Count of cleaned: 100


In [68]:
frame.head()

,month,day,year
0,03,25,93
1,06,18,85
2,07,08,71
3,09,27,75
4,02,06,96


In [69]:
frame.year = clean(frame.year, TWO_DIGITS, lambda match: "19" + match.group())

Random Sample Before:
44    71
72    95
55    87
38    94
41    75
Name: year, dtype: object

Random Sample After:
62    1987
6     1978
9     1971
35    1994
16    1977
Name: year, dtype: object

Count of cleaned: 100


In [70]:
twentieth_cleaned = frame_to_series(frame, twentieth)

     match
96   0        07/18/1986
81   0        08/04/1978
36   0        02/14/1973
114  0        12/08/1997
72   0        07/11/1977
dtype: object


In [71]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, twentieth_cleaned])

In [72]:
assert len(cleaned) == original + twentieth_count

In [73]:
print(numeric.head())

    match
14  0         5/24/1990
15  0         1/25/2011
17  0        10/13/1976
24  0        07/25/1984
30  0        03/31/1985
Name: numeric, dtype: object


In [74]:
has_dashes = numeric.str.contains(DASH)
print(numeric[has_dashes])

Series([], Name: numeric, dtype: object)


In [75]:
frame = pandas.DataFrame(numeric.str.split(SLASH).tolist(),
                         columns="month day year".split())
print(frame.head())

  month day  year
0     5  24  1990
1     1  25  2011
2    10  13  1976
3    07  25  1984
4    03  31  1985


In [76]:
frame.month = clean_two_digits_isolated(frame.month)

Random Sample Before:
18    04
10    12
20     4
16    09
3     07
Name: month, dtype: object

Random Sample After:
1     01
14    01
9     10
19    12
24    04
Name: month, dtype: object

Count of cleaned: 25


In [77]:
frame.day = clean_two_digits_isolated(frame.day)

Random Sample Before:
4     31
7     13
6     27
19    08
12    29
Name: day, dtype: object

Random Sample After:
6     27
10    05
3     25
16    14
8     15
Name: day, dtype: object

Count of cleaned: 25


In [78]:
numeric_cleaned = frame_to_series(frame, numeric)

     match
109  0        07/20/2011
38   0        07/27/1986
92   0        04/08/2004
34   0        05/12/2012
94   0        12/08/1990
dtype: object


In [79]:
original = len(cleaned)
cleaned = pandas.concat([cleaned, numeric_cleaned])
assert len(cleaned) == original + numeric_count
print(len(cleaned))

500


In [80]:
cleaned = pandas.concat([numeric_cleaned,
                         twentieth_cleaned,
                         words_cleaned,
                         backwards_cleaned,
                         no_day_cleaned,
                         no_day_numeric_cleaned,
                         year_only_cleaned,
                         leftovers_cleaned,
])
print(len(cleaned))
print(cleaned.head())
assert len(cleaned) == len(data)

500
    match
14  0        05/24/1990
15  0        01/25/2011
17  0        10/13/1976
24  0        07/25/1984
30  0        03/31/1985
dtype: object


In [81]:
print(cleaned.head())
datetimes = pandas.to_datetime(cleaned, format="%m/%d/%Y")
print(datetimes.head())

    match
14  0        05/24/1990
15  0        01/25/2011
17  0        10/13/1976
24  0        07/25/1984
30  0        03/31/1985
dtype: object
    match
14  0       1990-05-24
15  0       2011-01-25
17  0       1976-10-13
24  0       1984-07-25
30  0       1985-03-31
dtype: datetime64[ns]


In [82]:
sorted_dates = datetimes.sort_values()
print(sorted_dates.head())

    match
9   0       1971-04-10
84  0       1971-05-18
2   0       1971-07-08
53  0       1971-07-11
28  0       1971-09-12
dtype: datetime64[ns]


In [83]:
print(sorted_dates.tail())

     match
231  0       2016-05-01
141  0       2016-05-30
186  0       2016-10-13
161  0       2016-10-19
413  0       2016-11-01
dtype: datetime64[ns]


In [84]:
answer = pandas.Series(sorted_dates.index.labels[0])
print(answer.head())

0     9
1    84
2     2
3    53
4    28
dtype: int16


In [85]:
def date_sorter():
    return answer